In [135]:
import numpy as np

from skimage.transform import probabilistic_hough_line
from skimage.util import view_as_windows

from collections import namedtuple
import math
import random

from skimage.morphology import dilation, disk, square
from skimage import io
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from scipy import stats
import itertools
import functools
from skimage import filters, img_as_ubyte, img_as_float
from skimage import transform, exposure
from sklearn import metrics
from tqdm.notebook import tqdm
from skimage.measure import label, regionprops
from matplotlib import pyplot as plt

In [66]:
random.seed(10)
np.random.seed(10)

In [67]:
Point = namedtuple('Point', 'x y')
Segment = namedtuple('Segment', 'point_0, point_1, length')
mi_sigma = 1
points_dist = lambda pt_0, pt_1 : math.sqrt((pt_0.x-pt_1.x)**2 + (pt_0.y-pt_1.y)**2)
segment_angle = lambda segment : math.atan2((segment.point_1.y-segment.point_0.y), (segment.point_1.x-segment.point_0.x)) #* 180 / math.pi
window_size = 16
img_size = 512
step_factor = 1

In [68]:
def norm_counts(counts, weights):
    weights = [i/ min(weights) for i in weights]
    counts_norm = [list(itertools.repeat(count, int(weight)))  for count, weight in zip(counts, weights)]
    return functools.reduce(lambda x1, x2 : x1+x2, counts_norm) 

In [212]:
def joint_filter(image):
    image_f = img_as_float(image)
    coords = []
    for i in range(image.shape[0]-1):
        for j in range(image.shape[1]-1):
            if i > 1 and j > 1:
                # print(np.sum(image[i-1:i+2, j-1:j+2]))
                if image_f[i, j] == 1 and np.sum(image_f[i-1:i+2, j-1:j+2])>3:
                    coords.append(np.asarray([i, j]))
                    image[i, j] = 0    
    return np.vstack(coords) if len(coords)>0 else np.array([]), image

def compute_feats(test_patch, smooth_sigma=2):
    # compute cirvar, cirmean, lenvar, lenmean, num_segs, alignment coefficient (normalized)
    lines = probabilistic_hough_line(test_patch, threshold=1, line_length=2, line_gap=0, seed=0)
    angles = []
    lengths = []
    joints, filtered = joint_filter(test_patch)
    labeled = label(filtered, connectivity=2)
    regions = regionprops(labeled)
    line_lengths = np.asarray([region.area for region in regions if region.area>3])
    for line in lines:
        p0, p1 = line # (x1, y1), (x2, y2)
        if p0[0]<=p1[0]:
            point_0 = Point(p0[0], p0[1])
            point_1 = Point(p1[0], p1[1])
        if p0[0]>p1[0]:
            point_1 = Point(p0[0], p0[1])
            point_0 = Point(p1[0], p1[1])
        segment = Segment(point_0, point_1, points_dist(point_0, point_1))
        angles.append(segment_angle(segment))
        lengths.append(segment.length)
        
    density = exposure.rescale_intensity(filters.gaussian(dilation(test_patch, selem=disk(1)), sigma=smooth_sigma, preserve_range=False), out_range=(0, 1))
    if len(lines)>0:
        angles_norm = norm_counts(angles, lengths)
        cirmean = stats.circmean(angles_norm, high=math.pi/2, low=-math.pi/2)
        cirvar = stats.circvar(angles_norm, high=math.pi/2, low=-math.pi/2)
        lenmean = np.mean(line_lengths) + joints.shape[0]/np.count_nonzero(test_patch)
        lenvar = np.var(line_lengths/lenmean)
        intensity = np.count_nonzero(test_patch)
        feats = {'cir_mean' : cirmean, 'cir_var' : cirvar, 'len_mean' : lenmean, 'len_var' : lenvar, 'intensity' : intensity, 'density' : density}
    else:
        feats = {'cir_mean' : 1, 'cir_var' : 1, 'len_mean' : 1, 'len_var' : 1, 'intensity' : 1, 'density' : density}
    return feats

In [197]:
def patch_sequence_feats(sequence_patches):
    feats = []
    for i in range(sequence_patches.shape[0]):
        for j in range(sequence_patches.shape[1]):
            test_patch = sequence_patches[i, j]
            feats.append(compute_feats(test_patch))
    return feats

In [198]:
def image_to_feats(edges, step_factor=8):
    edges = edges.copy()
    # edges_padded = np.pad(edges, pad_width=(int(window_size/2), int(window_size/2)), mode='reflect')
    sequence_patches = view_as_windows(edges, window_shape=(window_size, window_size), step=(int(window_size/step_factor), int(window_size/step_factor)))
    feats = patch_sequence_feats(sequence_patches)
    return feats, (sequence_patches.shape[0], sequence_patches.shape[1])

In [199]:
def iou(mask_1, mask_2, beta=1e-3, soft=False):
    if soft:
        union = (mask_1**2 + mask_2**2)/2
        intersection = mask_1 * mask_2
    else:
        union = np.logical_or(mask_1, mask_2)
        intersection = np.logical_and(mask_1, mask_2)
    ratio = (intersection.sum()+beta)/(union.sum()+beta)
    return ratio


def norm_feats(vecs):
    vecs_1 = vecs.copy()
    for col in range(vecs_1.shape[1]):
        vecs_1[:, col] = exposure.rescale_intensity(vecs_1[:, col], out_range=(0, 1))
    return vecs_1

In [73]:
edges_1 = io.imread('Michael-1.tif')
edges_2 = io.imread('Bin-1.tif')

In [74]:
feats_1, shape = image_to_feats(edges_1, step_factor)
feats_2, _ = image_to_feats(edges_2, step_factor)
iou_scores = []
vecs_1 = []
vecs_2 = []
for feat_1, feat_2 in zip(feats_1, feats_2):
    vecs_1.append(np.array( [ feat_1[k] for k in list(feat_1.keys())[:-1] ] ))
    vecs_2.append(np.array( [ feat_2[k] for k in list(feat_2.keys())[:-1] ] ))
    iou_scores.append(iou(feat_1['density'], feat_2['density'], soft=True))
vecs_1 = np.vstack(vecs_1)
vecs_2 = np.vstack(vecs_2)

In [75]:
vecs_1_norm = norm_feats(vecs_1)
vecs_2_norm = norm_feats(vecs_2)
sims = []
for vec_1, vec_2 in zip(vecs_1_norm, vecs_2_norm):
    sim = metrics.pairwise.cosine_similarity(vec_1.reshape(1, -1), vec_2.reshape(1, -1))
    # sim = metrics.mean_squared_error(vec_1.reshape(1, -1), vec_2.reshape(1, -1)) * (vec_1[-1]+vec_2[-1])/2
    sims.append(sim)
# sims = exposure.rescale_intensity( np.array([max(sims)-i for i in sims]), out_range=(0, 1))

In [76]:
canvas = np.zeros(shape[0]*shape[1])
for idx, item in enumerate(sims):
    canvas[idx] = 0.8*item + 0.2*iou_scores[idx]

In [77]:
canvas = canvas.reshape(shape)
canvas = 1- transform.resize(canvas, (int(shape[0]*window_size/step_factor), int(shape[1]*window_size/step_factor)))

In [78]:
io.imsave('test-soft.tif', img_as_ubyte(canvas))